In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
tf.config.set_visible_devices([], 'GPU')

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img_data = np.asarray(img)
            images.append(img_data)
    return images

In [3]:
def load_images_from_folder_2(folder):
    images = []
    for filename in os.listdir(folder):
        loaded_img = load_img(os.path.join(folder,filename), target_size=(150, 150))
        numpy_img = img_to_array(loaded_img)
        images.append(numpy_img)
    return images

In [4]:
control_images = load_images_from_folder_2('CT_NonCOVID')
covid_images = load_images_from_folder_2('CT_COVID')
all_images = control_images + covid_images
all_images = np.array(all_images)
# all_images = all_images.reshape((-1, 150, 150, 3))

In [5]:

all_images.shape

(746, 150, 150, 3)

In [6]:
# for i in range(4):
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(np.uint8(all_images[i]))
# plt.show

In [7]:
control_labels = [0] * len(control_images)
covid_labels = [1] * len(covid_images)
label_series = pd.Series(control_labels + covid_labels)
print(len(all_images))
print(len(label_series))

746
746


In [8]:
X_train, X_test, y_train, y_test = train_test_split(all_images, label_series, test_size=0.25, random_state=42)

In [9]:
print(len(X_train))
print(len(y_train))

559
559


In [10]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, batch_size=64, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Simple network 1'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 5s 9ms/sample - loss: 25.5777 - accuracy: 0.5331
Epoch 2/8
559/559 [==============================] - 5s 8ms/sample - loss: 10.2801 - accuracy: 0.5116
Epoch 3/8
559/559 [==============================] - 5s 8ms/sample - loss: 4.1554 - accuracy: 0.5689
Epoch 4/8
559/559 [==============================] - 5s 8ms/sample - loss: 2.0256 - accuracy: 0.6011
Epoch 5/8
559/559 [==============================] - 5s 8ms/sample - loss: 1.1150 - accuracy: 0.6673
Epoch 6/8
559/559 [==============================] - 5s 8ms/sample - loss: 0.5912 - accuracy: 0.7478
Epoch 7/8
559/559 [==============================] - 6s 10ms/sample - loss: 0.4758 - accuracy: 0.7710
Epoch 8/8
559/559 [==============================] - 5s 8ms/sample - loss: 0.3254 - accuracy: 0.8479
Simple network 1 - accuracy: 0.7433155
Simple network 1 - recall: 0.5714286
Simple network 1 - precision: 0.7457627
Simple network 1 - F1 score: 0.6470588182057273


In [12]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, batch_size=64, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Simple network 2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 18s 32ms/sample - loss: 6.6913 - accuracy: 0.5259
Epoch 2/8
559/559 [==============================] - 17s 31ms/sample - loss: 1.3875 - accuracy: 0.5081
Epoch 3/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.8167 - accuracy: 0.5564
Epoch 4/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.7208 - accuracy: 0.5617
Epoch 5/8
559/559 [==============================] - 19s 33ms/sample - loss: 0.6410 - accuracy: 0.6154
Epoch 6/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.6317 - accuracy: 0.6279
Epoch 7/8
559/559 [==============================] - 19s 34ms/sample - loss: 0.5540 - accuracy: 0.6977
Epoch 8/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.5260 - accuracy: 0.7335
Simple network 2 - accuracy: 0.7486631
Simple network 2 - recall: 0.5714286
Simple network 2 - precision: 0.7586207
Simple network 2 - F1 score: 0.65185186

In [14]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\bruce\miniconda3\envs\capstone\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [15]:
model.fit(X_train, y_train, batch_size=32, epochs=10, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'Mobile Net V2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 14s 24ms/sample - loss: 0.6830 - accuracy: 0.5778
Epoch 2/8
559/559 [==============================] - 13s 23ms/sample - loss: 0.5326 - accuracy: 0.7370
Epoch 3/8
559/559 [==============================] - 12s 22ms/sample - loss: 0.4576 - accuracy: 0.7692
Epoch 4/8
559/559 [==============================] - 13s 23ms/sample - loss: 0.4047 - accuracy: 0.8336
Epoch 5/8
559/559 [==============================] - 12s 22ms/sample - loss: 0.3724 - accuracy: 0.8479
Epoch 6/8
559/559 [==============================] - 13s 23ms/sample - loss: 0.3686 - accuracy: 0.8408
Epoch 7/8
559/559 [==============================] - 13s 23ms/sample - loss: 0.3236 - accuracy: 0.8891
Epoch 8/8
559/559 [==============================] - 14s 25ms/sample - loss: 0.2966 - accuracy: 0.8945
Mobile Net V2 - accuracy: 0.5080214
Mobile Net V2 - recall: 0.36363637
Mobile Net V2 - precision: 0.3943662
Mobile Net V2 - F1 score: 0.37837839431766795


In [17]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.ResNet50V2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size=32, epochs=10, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'ResNet50 V2'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 21s 38ms/sample - loss: 0.6641 - accuracy: 0.5957
Epoch 2/8
559/559 [==============================] - 20s 35ms/sample - loss: 0.5065 - accuracy: 0.7567
Epoch 3/8
559/559 [==============================] - 19s 34ms/sample - loss: 0.4398 - accuracy: 0.8175
Epoch 4/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.4179 - accuracy: 0.8354
Epoch 5/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.3563 - accuracy: 0.8748
Epoch 6/8
559/559 [==============================] - 20s 36ms/sample - loss: 0.3215 - accuracy: 0.8998
Epoch 7/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.2879 - accuracy: 0.9177
Epoch 8/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.2741 - accuracy: 0.9141
ResNet50 V2 - accuracy: 0.4117647
ResNet50 V2 - recall: 1.0
ResNet50 V2 - precision: 0.4117647
ResNet50 V2 - F1 score: 0.583333326296674


In [19]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.VGG16(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, batch_size=32, epochs=10, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'VGG16'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 17s 30ms/sample - loss: 1.9978 - accuracy: 0.4937
Epoch 2/8
559/559 [==============================] - 17s 31ms/sample - loss: 1.1332 - accuracy: 0.5993
Epoch 3/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.6868 - accuracy: 0.7120
Epoch 4/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.5151 - accuracy: 0.7961
Epoch 5/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.3918 - accuracy: 0.8390
Epoch 6/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.3189 - accuracy: 0.8676
Epoch 7/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.2656 - accuracy: 0.8945
Epoch 8/8
559/559 [==============================] - 19s 33ms/sample - loss: 0.2206 - accuracy: 0.9231
VGG16 - accuracy: 0.684492
VGG16 - recall: 0.71428573
VGG16 - precision: 0.59782606
VGG16 - F1 score: 0.6508875156358002


In [21]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

base_model = tf.keras.applications.VGG19(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=32, epochs=10, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()

network_type = 'VGG19'
print(network_type, '-', 'accuracy:', ba.result().numpy())
print(network_type, '-', 'recall:', recall_result)
print(network_type, '-', 'precision:', precision_result)
print(network_type, '-', 'F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 21s 37ms/sample - loss: 1.2177 - accuracy: 0.5635
Epoch 2/8
559/559 [==============================] - 21s 37ms/sample - loss: 0.8045 - accuracy: 0.6995
Epoch 3/8
559/559 [==============================] - 21s 38ms/sample - loss: 0.5874 - accuracy: 0.7621
Epoch 4/8
559/559 [==============================] - 21s 38ms/sample - loss: 0.4471 - accuracy: 0.8068
Epoch 5/8
559/559 [==============================] - 22s 39ms/sample - loss: 0.3486 - accuracy: 0.8444
Epoch 6/8
559/559 [==============================] - 21s 38ms/sample - loss: 0.3052 - accuracy: 0.8605
Epoch 7/8
559/559 [==============================] - 21s 37ms/sample - loss: 0.2481 - accuracy: 0.9052
Epoch 8/8
559/559 [==============================] - 21s 37ms/sample - loss: 0.2241 - accuracy: 0.9195
VGG19 - accuracy: 0.7112299
VGG19 - recall: 0.58441556
VGG19 - precision: 0.67164177
VGG19 - F1 score: 0.625
